<a href="https://colab.research.google.com/github/chengwen-zheng/openAi-lab/blob/main/LangChain_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install langchain
%pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 666.0/666.0 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 1.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 3.0 MB/s eta 0:00:00


# 多次调用完成中译英，英文文答，英译中

In [3]:
from googleapiclient.discovery import build
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import io
import json
import os
import openai

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()

# Build the Drive API client
drive_service = build('drive', 'v3')
file_id = '14x1k7ErttIPXRcKN53g_Ogn3zfn6AEM_'
file = drive_service.files().get(fileId=file_id).execute()
content = drive_service.files().get_media(fileId=file_id).execute()

# Read the env config.json. set env for OPENAI_API_KEY
with io.BytesIO(content) as f:
    config = json.load(f)
os.environ["OPENAI_API_KEY"] = config["api_key"];
openai.api_key = os.environ["OPENAI_API_KEY"] ;

In [4]:
import openai, os
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain

openai.api_key = os.environ.get("OPENAI_API_KEY")

llm = OpenAI(model_name="text-davinci-003", max_tokens=2048, temperature=0.5)

en_to_zh_prompt = PromptTemplate(
    template="请把下面这句话翻译成英文： \n\n {question}?", input_variables=["question"]
)

question_prompt = PromptTemplate(
    template = "{english_question}", input_variables=["english_question"]
)

zh_to_cn_prompt = PromptTemplate(
    input_variables=["english_answer"],
    template="请把下面这一段翻译成中文： \n\n{english_answer}?",
)


question_translate_chain = LLMChain(llm=llm, prompt=en_to_zh_prompt, output_key="english_question")
english = question_translate_chain.run(question="请你作为一个机器学习的专家，介绍一下CNN的原理。")
print(english)

qa_chain = LLMChain(llm=llm, prompt=question_prompt, output_key="english_answer")
english_answer = qa_chain.run(english_question=english)
print(english_answer)

answer_translate_chain = LLMChain(llm=llm, prompt=zh_to_cn_prompt)
answer = answer_translate_chain.run(english_answer=english_answer)
print(answer)



Please explain the principle of CNN as an expert in machine learning.


A Convolutional Neural Network (CNN) is a type of artificial neural network that is used for image recognition and classification. It is a deep learning algorithm that uses a set of convolutional layers to extract features from an image, and then uses a fully connected layer to classify the image. The convolutional layers are used to detect patterns in the image, such as edges, shapes, and textures. The fully connected layer is then used to classify the image based on the patterns it has detected. CNNs are used in a wide range of applications, such as object detection, image segmentation, and facial recognition.


卷积神经网络（CNN）是一种用于图像识别和分类的人工神经网络。它是一种深度学习算法，使用一组卷积层从图像中提取特征，然后使用完全连接层对图像进行分类。卷积层用于检测图像中的模式，如边缘，形状和纹理。然后使用完全连接层根据检测到的模式对图像进行分类。CNN在许多应用中使用，如物体检测，图像分割和面部识别。


# 使用Langchain简化实现

In [5]:

from langchain.chains import SimpleSequentialChain
chinese_qa_chain = SimpleSequentialChain(
    chains=[question_translate_chain, qa_chain, answer_translate_chain], input_key="question",
    verbose=True)
answer = chinese_qa_chain.run(question="请你作为一个机器学习的专家，介绍一下CNN的原理。")
print(answer)



> Entering new SimpleSequentialChain chain...


Please introduce the principle of CNN as an expert in machine learning.


Convolutional Neural Networks (CNNs) are a type of neural network used in deep learning. They are inspired by the structure of the human brain and are used to detect patterns in large datasets. CNNs are used for image recognition, natural language processing, and various other applications in the field of machine learning. The basic principle of CNNs is to detect features in an image and then classify them. This is done by using convolutional layers which apply a filter to the input image, creating feature maps which are then used to detect patterns in the image. The layers are then connected to a fully connected layer which is used to classify the input image.


卷积神经网络（CNNs）是深度学习中使用的一种神经网络。它们受到人脑结构的启发，用于检测大型数据集中的模式。CNNs用于图像识别、自然语言处理和机器学习领域的各种其他应用。CNNs的基本原理是检测图像中的特征，然后对其进行分类。这是通过使用卷积层来实现的，它将滤波器应用于输入图像，从而创建特征映射，然后用于检测图像中的模式。然后将这些层连接到完全连接的层，用于对输入图像进行分类。

> Finished 

# 多步骤引入不同输入参数

In [6]:
from langchain.chains import SequentialChain

q1_prompt = PromptTemplate(
    input_variables=["year1"],
    template="{year1}年的欧冠联赛的冠军是哪支球队，只说球队名称。"
)
q2_prompt = PromptTemplate(
    input_variables=["year2"],
    template="{year2}年的欧冠联赛的冠军是哪支球队，只说球队名称。"
)
q3_prompt = PromptTemplate(
    input_variables=["team1", "team2"],
    template="{team1}和{team2}哪只球队获得欧冠的次数多一些？"
)
chain1 = LLMChain(llm=llm, prompt=q1_prompt, output_key="team1")
chain2 = LLMChain(llm=llm, prompt=q2_prompt, output_key="team2")
chain3 = LLMChain(llm=llm, prompt=q3_prompt)

sequential_chain = SequentialChain(chains=[chain1, chain2, chain3], input_variables=["year1", "year2"], verbose=True)
answer = sequential_chain.run(year1=2000, year2=2010)
print(answer)



> Entering new SequentialChain chain...

> Finished chain.


拜仁慕尼黑获得欧冠的次数更多，共计8次，而法国巴黎圣日耳曼队只有6次。


# 通过Langchain来完成自动撰写单元测试


In [7]:
from langchain.chains import SequentialChain


def write_unit_test(function_to_test, unit_test_package = "pytest"):
    # 解释源代码的步骤
    explain_code = """"# How to write great unit tests with {unit_test_package}

    In this advanced tutorial for experts, we'll use Python 3.10 and `{unit_test_package}` to write a suite of unit tests to verify the behavior of the following function.
    ```python
    {function_to_test}
    ```

    Before writing any unit tests, let's review what each element of the function is doing exactly and what the author's intentions may have been.
    - First,"""

    explain_code_template = PromptTemplate(
        input_variables=["unit_test_package", "function_to_test"],
        template=explain_code
    )
    explain_code_llm = OpenAI(model_name="text-davinci-002", temperature=0.4, max_tokens=1000, 
            top_p=1, stop=["\n\n", "\n\t\n", "\n    \n"])
    explain_code_step = LLMChain(llm=explain_code_llm, prompt=explain_code_template, output_key="code_explaination")


    # 创建测试计划示例的步骤
    test_plan = """
        
    A good unit test suite should aim to:
    - Test the function's behavior for a wide range of possible inputs
    - Test edge cases that the author may not have foreseen
    - Take advantage of the features of `{unit_test_package}` to make the tests easy to write and maintain
    - Be easy to read and understand, with clean code and descriptive names
    - Be deterministic, so that the tests always pass or fail in the same way

    `{unit_test_package}` has many convenient features that make it easy to write and maintain unit tests. We'll use them to write unit tests for the function above.

    For this particular function, we'll want our unit tests to handle the following diverse scenarios (and under each scenario, we include a few examples as sub-bullets):
    -"""
    test_plan_template = PromptTemplate(
        input_variables=["unit_test_package", "function_to_test", "code_explaination"],
        template= explain_code + "{code_explaination}" + test_plan
    )
    test_plan_llm = OpenAI(model_name="text-davinci-002", temperature=0.4, max_tokens=1000, 
            top_p=1, stop=["\n\n", "\n\t\n", "\n    \n"])
    test_plan_step = LLMChain(llm=test_plan_llm, prompt=test_plan_template, output_key="test_plan")

    # 撰写测试代码的步骤
    starter_comment = "Below, each test case is represented by a tuple passed to the @pytest.mark.parametrize decorator"
    prompt_to_generate_the_unit_test = """

Before going into the individual tests, let's first look at the complete suite of unit tests as a cohesive whole. We've added helpful comments to explain what each line does.
```python
import {unit_test_package}  # used for our unit tests

{function_to_test}

#{starter_comment}"""

    unit_test_template = PromptTemplate(
        input_variables=["unit_test_package", "function_to_test", "code_explaination", "test_plan", "starter_comment"],
        template= explain_code + "{code_explaination}" + test_plan + "{test_plan}" + prompt_to_generate_the_unit_test
    )
    unit_test_llm = OpenAI(model_name="text-davinci-002", temperature=0.4, max_tokens=1000, stop="```")
    unit_test_step = LLMChain(llm=unit_test_llm, prompt=unit_test_template, output_key="unit_test")

    sequential_chain = SequentialChain(chains=[explain_code_step, test_plan_step, unit_test_step], 
                                    input_variables=["unit_test_package", "function_to_test", "starter_comment"], verbose=True)
    answer = sequential_chain.run(unit_test_package=unit_test_package, function_to_test=function_to_test, starter_comment=starter_comment)
    return f"""#{starter_comment}""" + answer


code = """
def format_time(seconds):
    minutes, seconds = divmod(seconds, 60)
    hours, minutes = divmod(minutes, 60)
    if hours > 0:
        return f"{hours}h{minutes}min{seconds}s"
    elif minutes > 0:
        return f"{minutes}min{seconds}s"
    else:
        return f"{seconds}s"
"""

import ast

def write_unit_test_automatically(code, retry=3):
    unit_test_code = write_unit_test(code)
    all_code = code + unit_test_code
    tried = 0
    while tried < retry:
        try:
            ast.parse(all_code)
            return all_code
        except SyntaxError as e:
            print(f"Syntax error in generated code: {e}")
            all_code = code + write_unit_test(code)
            tried += 1

print(write_unit_test_automatically(code))

                    stop was transfered to model_kwargs.
                    Please confirm that stop is what you intended.
                    stop was transfered to model_kwargs.
                    Please confirm that stop is what you intended.
                    stop was transfered to model_kwargs.
                    Please confirm that stop is what you intended.




> Entering new SequentialChain chain...

> Finished chain.

def format_time(seconds):
    minutes, seconds = divmod(seconds, 60)
    hours, minutes = divmod(minutes, 60)
    if hours > 0:
        return f"{hours}h{minutes}min{seconds}s"
    elif minutes > 0:
        return f"{minutes}min{seconds}s"
    else:
        return f"{seconds}s"
#Below, each test case is represented by a tuple passed to the @pytest.mark.parametrize decorator.
#The first element of the tuple is the name of the test case, and the second element is a list of
#the inputs to be passed to the function under test.
@pytest.mark.parametrize(
    "test_case,seconds",
    [
        # Test the function's behavior for a wide range of possible inputs
        ("normal behavior", [0, 1, 59, 60, 61, 3599, 3600, 3601, 3660, 5400, 5401]),
        # Test edge cases that the author may not have foreseen
        ("edge cases", [60.1, 3600.1, 5400.1]),
    ],
)
def test_format_time(test_case, seconds):
    for second in seconds:
  